In [ ]:
!pip install opencv-python
!pip install numpy
!pip install torch

In [247]:
import csv
import cv2
import numpy as np
import random
import os

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [248]:
TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
TEST_PATH = "/kaggle/input/captcha-hacker/test"
# device = "cpu"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [249]:
train_data = []
val_data = []
test_data = []

with open(f'{TRAIN_PATH}/annotations.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.7:
            train_data.append(row)
        else:
            val_data.append(row)

with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

In [250]:
def horizonFlip(img):
    flip_img = np.zeros(img.shape, dtype=np.uint8)
    row_idx = 0
    for row in img:
        flip_img[row_idx] = row[::-1]
        row_idx += 1
    return flip_img

def colorJittering(img):
    color_img = np.zeros(img.shape, dtype=np.uint8)
    for r in range(len(img)):
        for c in range(len(img[r])):
            color_img[r][c][:2] = img[r][c][:2]  # Turn off 'B' channel
    return color_img

def noiseInjection(img):
    noise = np.random.normal(0, 8, img.shape)
    noise = np.array([int(n) for row in noise for col in row for n in col])
    noise = noise.reshape(img.shape)
    noise_img = img + noise
    noise_img[np.where(noise_img > 255)] = 255
    noise_img[np.where(noise_img < 0)] = 0
    return noise_img

def dataAugmentation(x, y):
    aug_number = 4
    x_new = np.zeros(shape=(len(x) * aug_number, 32, 32, 1), dtype=np.uint8)
    y_new = np.zeros(shape=(len(y) * aug_number, len(y[0])), dtype=np.int64)
    for i in range(0, len(x) * aug_number, aug_number):
        index = i // aug_number
        y_new[i: i + aug_number] = y[index]
        
        #print(y_new[i: i + aug_number].shape) = (4, 1751)
        #print(y[index].shape) = (2,)
        
        x_new[i] = x[index]
        x_new[i + 1] = horizonFlip(x[index])
        x_new[i + 2] = colorJittering(x[index])
        x_new[i + 3] = noiseInjection(x[index])
    return x_new, y_new

In [251]:
LETTERSTR = "0123456789abcdefghijklmnopqrstuvwxyz"

def encode(text):
    labellist = []
    for letter in text:
        num = LETTERSTR.find(letter)
        labellist.append(num)
    return labellist

LETTERSTR_char = "0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z"
LETTERSTR_char = LETTERSTR_char.split(",")
LETTERSTR_index = [i for i in range(36)]
decode = dict(zip(LETTERSTR_index, LETTERSTR_char))

In [252]:
def ExtractData(data, root, task_round):
    filenames = np.array([sample for sample in data if sample[0].startswith(task_round)])
    imgs_names = filenames[:, 0]
    label = filenames[:, 1]
    imgs = np.zeros(shape=(len(filenames), 32, 32, 1))
    labels = np.zeros(shape=(len(filenames), len(label[0])))
    for i in range(len(filenames)):
        tmp = cv2.imread(f"{root}/{imgs_names[i]}")
        tmp = cv2.resize(tmp, (32, 32))
        tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
        imgs[i] = np.expand_dims(tmp, axis=2)
        tmp_label = encode(label[i])
        labels[i] = np.array(tmp_label)
    return imgs_names, imgs, labels

class TaskDataset(Dataset):
    def __init__(self, x_data, y_data, filenames, return_filename=False):
        x_data = x_data.astype('float32')
        x_data = x_data / 255.0
        self.x_data = torch.from_numpy(x_data).permute(0, 3, 1, 2) # (N, 1, 32, 32)
        self.y_data = torch.from_numpy(y_data) # (N, 1)
        self.filenames = filenames
        self.return_filename = return_filename
        
    def __getitem__(self, index):
        if self.return_filename:
            return self.x_data[index], self.filenames[index] # aug_num = 5
        else:
            return self.x_data[index], self.y_data[index]
        
    def __len__(self):
        return len(self.x_data)

In [253]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (32, 32))
        
        img = torch.tensor(img).permute(2,0,1)
        if self.return_filename:
            return torch.FloatTensor((img - 128) / 128), filename
        else:
            return torch.FloatTensor((img - 128) / 128), int(label)

    def __len__(self):
        return len(self.data)

In [254]:
# task1 data processing
train_ds = Task1Dataset(train_data, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=40, num_workers=2, drop_last=True, shuffle=True)

val_ds = Task1Dataset(val_data, root=TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=40, num_workers=2, drop_last=False, shuffle=False)

test_ds = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=130, num_workers=2, drop_last=False, shuffle=False)

In [255]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        # input size = (3, 32, 32)
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size=3,
                                  stride=1, padding=1),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(64, 64, kernel_size=3,
                                  stride=1, padding=1),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)  # (64,16,16)
        self.dropout1 = nn.Dropout(p=0.2)

        self.conv3 = nn.Sequential(
                        nn.Conv2d(64, 128, kernel_size=3,
                                  stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU())
        self.conv4 = nn.Sequential(
                        nn.Conv2d(128, 128, kernel_size=3,
                                  stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU())
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)  # (128,8,8)
        self.dropout2 = nn.Dropout(p=0.3)

        self.conv5 = nn.Sequential(
                        nn.Conv2d(128, 128, kernel_size=3,
                                  stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU())
        self.conv6 = nn.Sequential(
                        nn.Conv2d(128, 128, kernel_size=3,
                                  stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU())
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)  # (128,4,4)
        self.dropout3 = nn.Dropout(p=0.4)

        # FC, input size = (16, 5, 5)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.relu3 = nn.ReLU()
        self.dropout4 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(512, 10)
        self.output = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.dropout1(self.maxpool1(self.conv2(self.conv1(x))))
        out = self.dropout2(self.maxpool2(self.conv4(self.conv3(out))))
        out = self.dropout3(self.maxpool3(self.conv6(self.conv5(out))))
        out = torch.flatten(out, 1)  # from CNN to FCN
        out = self.output(self.fc2(self.dropout4(self.relu3(self.fc1(out)))))
        return out

In [263]:
model = Model1().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


for epoch in range(100):
    print(f"Epoch [{epoch}]")
    model.train()
    for image, label in train_dl:
        image = image.to(device)
        label = label.to(device)
        
        pred = model(image)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    sample_count = 0
    correct_count = 0
    model.eval()
    for image, label in val_dl:
        image = image.to(device)
        label = label.to(device)
        
        pred = model(image)
        loss = loss_fn(pred, label)
        
        pred = torch.argmax(pred, dim=1)
        
        sample_count += len(image)
        correct_count += (label == pred).sum()
        
    print("accuracy (validation):", correct_count / sample_count)

Epoch [0]
accuracy (validation): tensor(0.0801, device='cuda:0')
Epoch [1]
accuracy (validation): tensor(0.1083, device='cuda:0')
Epoch [2]
accuracy (validation): tensor(0.2810, device='cuda:0')
Epoch [3]
accuracy (validation): tensor(0.4113, device='cuda:0')
Epoch [4]
accuracy (validation): tensor(0.5400, device='cuda:0')
Epoch [5]
accuracy (validation): tensor(0.5950, device='cuda:0')
Epoch [6]
accuracy (validation): tensor(0.6358, device='cuda:0')
Epoch [7]
accuracy (validation): tensor(0.6452, device='cuda:0')
Epoch [8]
accuracy (validation): tensor(0.6358, device='cuda:0')
Epoch [9]
accuracy (validation): tensor(0.8226, device='cuda:0')
Epoch [10]
accuracy (validation): tensor(0.8493, device='cuda:0')
Epoch [11]
accuracy (validation): tensor(0.9199, device='cuda:0')
Epoch [12]
accuracy (validation): tensor(0.9011, device='cuda:0')
Epoch [13]
accuracy (validation): tensor(0.9231, device='cuda:0')
Epoch [14]
accuracy (validation): tensor(0.8932, device='cuda:0')
Epoch [15]
accuracy 

In [264]:
task1 = {'model': Model1(),
         'state_dict': model.state_dict(),
         'optimizer' : optimizer.state_dict()}

torch.save(task1, 'task1.pth')

In [265]:
# predict test1 write to csv
csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
csv_writer.writerow(["filename", "label"])

model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    
    pred = model(image)
    pred = torch.argmax(pred, dim=1)
    
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], str(pred[i].item())])

In [266]:
# task2 data processing
filename_train, x_train, y_train = ExtractData(train_data, root=TRAIN_PATH, task_round="task2")
x_train_aug, y_train_aug = dataAugmentation(x_train, y_train)
train_ds = TaskDataset(x_train_aug, y_train_aug, filename_train)
train_dl = DataLoader(train_ds, batch_size=200, num_workers=2, drop_last=True, shuffle=True)

filename_val, x_val, y_val = ExtractData(val_data, root=TRAIN_PATH, task_round="task2")
val_ds = TaskDataset(x_val, y_val, filename_val)
val_dl = DataLoader(val_ds, batch_size=200, num_workers=2, drop_last=False, shuffle=False)

filename_test, x_test, y_test = ExtractData(test_data, root=TEST_PATH, task_round="task2")
test_ds = TaskDataset(x_test, y_test, filename_test, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=125, num_workers=2, drop_last=False, shuffle=False)

In [267]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        # input size = (3, 32, 32)
        self.conv1 = self.conv(1, 64, 3, 1, 1)  # (64,32,32)
        self.conv2 = self.conv(64, 64, 3, 1, 1)  # (64,32,32)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)  # (64,16,16)
        self.dropout1 = nn.Dropout(p=0.2)

        self.conv3 = self.conv(64, 128, 3, 1, 1)  # (128,16,16)
        self.conv4 = self.conv(128, 128, 3, 1, 1)  # (128,16,16)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)  # (128,8,8)
        self.dropout2 = nn.Dropout(p=0.3)

        self.conv5 = self.conv(128, 128, 3, 1, 1)  # (128,8,8)
        self.conv6 = self.conv(128, 128, 3, 1, 1)  # (128,8,8)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)  # (128,4,4)
        self.dropout3 = nn.Dropout(p=0.4)

        # FC, input size = (16, 5, 5)
        self.fc11 = nn.Linear(128 * 4 * 4, 512)
        self.relu31 = nn.ReLU()
        self.dropout41 = nn.Dropout(p=0.5)
        self.fc21 = nn.Linear(512, 36*2)
        
        self.fc12 = nn.Linear(128 * 4 * 4, 512)
        self.relu32 = nn.ReLU()
        self.dropout42 = nn.Dropout(p=0.5)
        self.fc22 = nn.Linear(512, 36*2)
        self.output = nn.Softmax(dim=1)

    def conv(self, in_c, out_c, kernel_size, stride=1, padding=0):
        conv_layer = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=kernel_size,
                      stride=stride, padding=padding),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )
        return conv_layer

    def forward(self, x):
        out = self.dropout1(self.maxpool1(self.conv2(self.conv1(x))))
        out = self.dropout2(self.maxpool2(self.conv4(self.conv3(out))))
        out = self.dropout3(self.maxpool3(self.conv6(self.conv5(out))))
        out = torch.flatten(out, 1)  # from CNN to FCN
        out1 = self.output(self.fc21(self.dropout41(self.relu31(self.fc11(out)))))
        out2 = self.output(self.fc22(self.dropout42(self.relu32(self.fc12(out)))))
        return out1, out2

In [299]:
model = Model2().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


for epoch in range(100):
    print(f"Epoch [{epoch}]")
    model.train()
    for image, label in train_dl:
        image = image.to(device)
        label = label.to(device)
        
        pred = model(image)
        
        label = label.long()
        loss1 = loss_fn(pred[0], label[:,0])
        loss2 = loss_fn(pred[1], label[:,1])
        loss = loss1 + loss2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    sample_count = 0
    correct_count = 0
    model.eval()
    for image, label in val_dl:
        image = image.to(device)
        label = label.to(device)
        
        pred = model(image)
        
        pred1 = torch.argmax(pred[0], dim=1)
        pred2 = torch.argmax(pred[1], dim=1)
        
        sample_count += 2*len(image)
        correct_count += (label[:,0] == pred1).sum()
        correct_count += (label[:,1] == pred2).sum()
        
    print("accuracy (validation):", correct_count / sample_count)

Epoch [0]
accuracy (validation): tensor(0.0627, device='cuda:0')
Epoch [1]
accuracy (validation): tensor(0.0904, device='cuda:0')
Epoch [2]
accuracy (validation): tensor(0.1286, device='cuda:0')
Epoch [3]
accuracy (validation): tensor(0.1201, device='cuda:0')
Epoch [4]
accuracy (validation): tensor(0.1570, device='cuda:0')
Epoch [5]
accuracy (validation): tensor(0.1992, device='cuda:0')
Epoch [6]
accuracy (validation): tensor(0.2619, device='cuda:0')
Epoch [7]
accuracy (validation): tensor(0.2685, device='cuda:0')
Epoch [8]
accuracy (validation): tensor(0.2856, device='cuda:0')
Epoch [9]
accuracy (validation): tensor(0.3443, device='cuda:0')
Epoch [10]
accuracy (validation): tensor(0.3621, device='cuda:0')
Epoch [11]
accuracy (validation): tensor(0.3806, device='cuda:0')
Epoch [12]
accuracy (validation): tensor(0.4004, device='cuda:0')
Epoch [13]
accuracy (validation): tensor(0.3964, device='cuda:0')
Epoch [14]
accuracy (validation): tensor(0.4162, device='cuda:0')
Epoch [15]
accuracy 

In [300]:
# predict test2 write to csv
if os.path.exists('submission.csv'):
    csv_writer = csv.writer(open('submission.csv', 'a', newline=''))
else:
    csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
    csv_writer.writerow(["filename", "label"])


model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    
    pred = model(image)
    pred1 = torch.argmax(pred[0], dim=1)
    pred2 = torch.argmax(pred[1], dim=1)
    for i in range(len(filenames)):
        cur_string = decode[pred1[i].item()]+decode[pred2[i].item()]
        csv_writer.writerow([filenames[i], cur_string])

In [301]:
task2 = {'model': Model2(),
         'state_dict': model.state_dict(),
         'optimizer' : optimizer.state_dict()}

torch.save(task2, 'task2.pth')

In [302]:
# task3 data processing
filename_train, x_train, y_train = ExtractData(train_data, root=TRAIN_PATH, task_round="task3")
x_train_aug, y_train_aug = dataAugmentation(x_train, y_train)
train_ds = TaskDataset(x_train_aug, y_train_aug, filename_train)
train_dl = DataLoader(train_ds, batch_size=240, num_workers=2, drop_last=True, shuffle=True)

filename_val, x_val, y_val = ExtractData(val_data, root=TRAIN_PATH, task_round="task3")
val_ds = TaskDataset(x_val, y_val, filename_val)
val_dl = DataLoader(val_ds, batch_size=240, num_workers=2, drop_last=False, shuffle=False)

filename_test, x_test, y_test = ExtractData(test_data, root=TEST_PATH, task_round="task3")
test_ds = TaskDataset(x_test, y_test, filename_test, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=20, num_workers=2, drop_last=False, shuffle=False)

In [303]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        # input size = (3, 32, 32)
        self.conv1 = self.conv(1, 64, 3, 1, 1)  # (64,32,32)
        self.conv2 = self.conv(64, 64, 3, 1, 1)  # (64,32,32)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)  # (64,16,16)
        self.dropout1 = nn.Dropout(p=0.2)

        self.conv3 = self.conv(64, 128, 3, 1, 1)  # (128,16,16)
        self.conv4 = self.conv(128, 128, 3, 1, 1)  # (128,16,16)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)  # (128,8,8)
        self.dropout2 = nn.Dropout(p=0.3)

        self.conv5 = self.conv(128, 128, 3, 1, 1)  # (128,8,8)
        self.conv6 = self.conv(128, 128, 3, 1, 1)  # (128,8,8)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)  # (128,4,4)
        self.dropout3 = nn.Dropout(p=0.4)

        # FC, input size = (16, 5, 5)
        self.fc11 = nn.Linear(128 * 4 * 4, 512)
        self.relu31 = nn.ReLU()
        self.dropout41 = nn.Dropout(p=0.5)
        self.fc21 = nn.Linear(512, 36*4)
        
        self.fc12 = nn.Linear(128 * 4 * 4, 512)
        self.relu32 = nn.ReLU()
        self.dropout42 = nn.Dropout(p=0.5)
        self.fc22 = nn.Linear(512, 36*4)
        
        self.fc13 = nn.Linear(128 * 4 * 4, 512)
        self.relu33 = nn.ReLU()
        self.dropout43 = nn.Dropout(p=0.5)
        self.fc23 = nn.Linear(512, 36*4)
        
        self.fc14 = nn.Linear(128 * 4 * 4, 512)
        self.relu34 = nn.ReLU()
        self.dropout44 = nn.Dropout(p=0.5)
        self.fc24 = nn.Linear(512, 36*4)
        self.output = nn.Softmax(dim=1)

    def conv(self, in_c, out_c, kernel_size, stride=1, padding=0):
        conv_layer = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=kernel_size,
                      stride=stride, padding=padding),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )
        return conv_layer

    def forward(self, x):
        out = self.dropout1(self.maxpool1(self.conv2(self.conv1(x))))
        out = self.dropout2(self.maxpool2(self.conv4(self.conv3(out))))
        out = self.dropout3(self.maxpool3(self.conv6(self.conv5(out))))
        out = torch.flatten(out, 1)  # from CNN to FCN
        out1 = self.output(self.fc21(self.dropout41(self.relu31(self.fc11(out)))))
        out2 = self.output(self.fc22(self.dropout42(self.relu32(self.fc12(out)))))
        out3 = self.output(self.fc23(self.dropout43(self.relu33(self.fc13(out)))))
        out4 = self.output(self.fc24(self.dropout44(self.relu34(self.fc14(out)))))
        return out1, out2, out3, out4

In [325]:
model = Model3().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


for epoch in range(100):
    print(f"Epoch [{epoch}]")
    model.train()
    for image, label in train_dl:
        image = image.to(device)
        label = label.to(device)
        
        pred = model(image)

        label = label.long()
        loss1 = loss_fn(pred[0], label[:,0])
        loss2 = loss_fn(pred[1], label[:,1])
        loss3 = loss_fn(pred[2], label[:,2])
        loss4 = loss_fn(pred[3], label[:,3])
        loss = loss1 + loss2 + loss3 + loss4
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    sample_count = 0
    correct_count = 0
    model.eval()
    for image, label in val_dl:
        image = image.to(device)
        label = label.to(device)
        
        pred = model(image)
        
        pred1 = torch.argmax(pred[0], dim=1)
        pred2 = torch.argmax(pred[1], dim=1)
        pred3 = torch.argmax(pred[2], dim=1)
        pred4 = torch.argmax(pred[3], dim=1)
        
        sample_count += 4*len(image)

        correct_count += (label[:,0] == pred1).sum()
        correct_count += (label[:,1] == pred2).sum()
        correct_count += (label[:,2] == pred3).sum()
        correct_count += (label[:,3] == pred4).sum()
        
    print("accuracy (validation):", correct_count / sample_count)

Epoch [0]
accuracy (validation): tensor(0.0392, device='cuda:0')
Epoch [1]
accuracy (validation): tensor(0.0567, device='cuda:0')
Epoch [2]
accuracy (validation): tensor(0.0588, device='cuda:0')
Epoch [3]
accuracy (validation): tensor(0.0703, device='cuda:0')
Epoch [4]
accuracy (validation): tensor(0.0781, device='cuda:0')
Epoch [5]
accuracy (validation): tensor(0.0858, device='cuda:0')
Epoch [6]
accuracy (validation): tensor(0.0982, device='cuda:0')
Epoch [7]
accuracy (validation): tensor(0.0976, device='cuda:0')
Epoch [8]
accuracy (validation): tensor(0.1115, device='cuda:0')
Epoch [9]
accuracy (validation): tensor(0.1175, device='cuda:0')
Epoch [10]
accuracy (validation): tensor(0.1259, device='cuda:0')
Epoch [11]
accuracy (validation): tensor(0.1302, device='cuda:0')
Epoch [12]
accuracy (validation): tensor(0.1365, device='cuda:0')
Epoch [13]
accuracy (validation): tensor(0.1411, device='cuda:0')
Epoch [14]
accuracy (validation): tensor(0.1570, device='cuda:0')
Epoch [15]
accuracy 

In [326]:
# predict task3 write to csv
csv_writer = csv.writer(open('submission.csv', 'a', newline=''))

model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    
    pred = model(image)
    pred1 = torch.argmax(pred[0], dim=1)
    pred2 = torch.argmax(pred[1], dim=1)
    pred3 = torch.argmax(pred[2], dim=1)
    pred4 = torch.argmax(pred[3], dim=1)
    
    for i in range(len(filenames)):
        cur_string = decode[pred1[i].item()]+decode[pred2[i].item()]+decode[pred3[i].item()]+decode[pred4[i].item()]
        csv_writer.writerow([filenames[i], cur_string])
        
    """
    for filename, value in filenames:
        cur_string = decode[pred1[i].item()]+decode[pred2[i].item()]+decode[pred3[i].item()]+decode[pred4[i].item()]
        csv_writer.writerow(filename, value)
    """

In [327]:
task3 = {'model': Model3(),
         'state_dict': model.state_dict(),
         'optimizer' : optimizer.state_dict()}

torch.save(task3, 'task3.pth')

In [328]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [329]:
model1 = load_checkpoint('task1.pth')
print(model1)
model2 = load_checkpoint('task2.pth')
print(model2)
model3 = load_checkpoint('task3.pth')
print(model3)

Model1(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.2, inplace=False)
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool2): MaxPool2d(ker